Make sure GPU is enabled
Runtime -> Change Runtime Type -> Hardware Accelerator -> GPU

In [ ]:
!pip install librosa==0.7.0

     |████████████████████████████████| 1.6MB 6.3MB/s 
  Created wheel for librosa: filename=librosa-0.7.0-cp37-none-any.whl size=1598345 sha256=6f2dba42d1fefb3f6dd9aefd8ebcd0ae1177cf8ca1ec4bac0394c0cef83517a0
  Stored in directory: /root/.cache/pip/wheels/49/1d/38/c8ad12fcad67569d8e730c3275be5e581bd589558484a0f881
Successfully built librosa
  Found existing installation: librosa 0.8.0
    Uninstalling librosa-0.8.0:
      Successfully uninstalled librosa-0.8.0


In [ ]:
pip install numba==0.48

     |████████████████████████████████| 3.6MB 6.4MB/s 
     |████████████████████████████████| 20.2MB 10.0MB/s 
  Found existing installation: llvmlite 0.34.0
    Uninstalling llvmlite-0.34.0:
      Successfully uninstalled llvmlite-0.34.0
  Found existing installation: numba 0.51.2
    Uninstalling numba-0.51.2:
      Successfully uninstalled numba-0.51.2


In [ ]:
!git clone https://github.com/vlomme/Multi-Tacotron-Voice-Cloning.git

Cloning into 'Multi-Tacotron-Voice-Cloning'...
remote: Enumerating objects: 238, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 238 (delta 0), reused 0 (delta 0), pack-reused 234
Receiving objects: 100% (238/238), 1009.42 KiB | 7.76 MiB/s, done.
Resolving deltas: 100% (92/92), done.


In [ ]:
cd Multi-Tacotron-Voice-Cloning/

/content/Multi-Tacotron-Voice-Cloning


In [ ]:
!pwd

/content/Multi-Tacotron-Voice-Cloning


You should creat new folder 'voices' in the Multi-Tacotron-Voice-Cloning folder

In [ ]:
%tensorflow_version 2.x
!pip uninstall -y tensorflow
!pip install tensorflow-gpu==1.14.0
!pip install -r requirements.txt
!pip install SpeechRecognition
!pip install pyTelegramBotAPI
!apt-get install libportaudio2

!gdown https://drive.google.com/uc?id=1aQBmpflbX_ePUdXTSNE4CfEL9hdG2-O8
!unzip pretrained.zip

Uninstalling tensorflow-2.4.1:
  Successfully uninstalled tensorflow-2.4.1
     |████████████████████████████████| 377.1MB 44kB/s 
     |████████████████████████████████| 491kB 38.7MB/s 
     |████████████████████████████████| 3.2MB 30.8MB/s 
     |████████████████████████████████| 51kB 6.4MB/s 
  Found existing installation: tensorflow-estimator 2.4.0
    Uninstalling tensorflow-estimator-2.4.0:
      Successfully uninstalled tensorflow-estimator-2.4.0
  Found existing installation: tensorboard 2.4.1
    Uninstalling tensorboard-2.4.1:
      Successfully uninstalled tensorboard-2.4.1
     |████████████████████████████████| 81kB 4.1MB/s 
     |████████████████████████████████| 686kB 8.9MB/s 
     |████████████████████████████████| 71kB 5.7MB/s 
     |████████████████████████████████| 245kB 11.2MB/s 
     |████████████████████████████████| 8.3MB 11.9MB/s 
     |████████████████████████████████| 184kB 39.1MB/s 
     |████████████████████████████████| 1.2MB 37.0MB/s 
     |███████████████

In [ ]:
from synthesizer.inference import Synthesizer
from encoder import inference as inference_encoder
from vocoder import inference as inference_vocoder
from g2p.train import g2p

/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

In [ ]:
from speech_recognition import (Recognizer, AudioFile)
from speech_recognition import (UnknownValueError, RequestError)
import torch

import telebot
from telebot import types

import requests
import time

from pathlib import Path
import subprocess
import sys
import os


import numpy as np
import librosa

In [ ]:
voices = [name.split('.')[0] for name in os.listdir('voices')]
voices.append("Записать свой")

In [ ]:
token_bot = "YOUR TOKEN"
bot = telebot.TeleBot(token_bot)

knownUsers = []  # todo: save these in a file,
userStep = {}  # so they won't reset every time the bot restarts

commands = {  # command description used in the "help" command
    'chooseVoice'    : "Выбрать или загрузить голос, который хотите склонировать",
    'reset': "Вернуться к началу"
}

voiceSelect = types.ReplyKeyboardMarkup(one_time_keyboard=True)  # create the image selection keyboard
voiceSelect.add(*voices)

hideBoard = types.ReplyKeyboardRemove()  # if sent as reply_markup, will hide the keyboard

In [ ]:
# error handling if user isn't known yet
# (obsolete once known users are saved to file, because all users
#   had to use the /start command and are therefore known to the bot)
def get_user_step(uid):
    if uid in userStep:
        return userStep[uid]
    else:
        knownUsers.append(uid)
        userStep[uid] = 0
        print("New user detected, who hasn't used \"/start\" yet")
        return 0


# only used for console output now
def listener(messages):
    for m in messages:
        if m.chat.id not in knownUsers:
            bot.send_message(m.chat.id, "Hello, stranger!")
        print("Got message of type", m.content_type)
        if m.content_type == 'text':
            print(str(m.chat.first_name) + " [" + str(m.chat.id) + "]: " + m.text)

        if m.content_type == 'voice':
            if userStep[m.chat.id] == 2:
                extract_target_voice(m)
            elif type(userStep[m.chat.id]) == str:
                generate_voice_message(m)

bot = telebot.TeleBot(token_bot)
bot.set_update_listener(listener)  # register listener


# handle the "/start" command
@bot.message_handler(commands=['start'])
def command_start(m):
    cid = m.chat.id
    if cid not in knownUsers: 
        if os.path.exists(str(cid)):
            clean(cid)
        else:
            os.mkdir(str(cid))

        knownUsers.append(cid) 
        userStep[cid] = 0  
    command_help(m)
    
@bot.message_handler(commands=['chooseVoice'])
def command_choose_voice(m):
    cid = m.chat.id
    bot.send_message(cid, "Выберите предзаписанные голос или запишите свой", reply_markup=voiceSelect)  # show the keyboard
    userStep[cid] = 1 

@bot.message_handler(commands=['reset'])
def command_choose_voice(m):
    cid = m.chat.id
    bot.send_message(cid, "Текущий голос сброшен")  # show the keyboard
    userStep[cid] = 0
    command_help(m) 

@bot.message_handler(func=lambda message: get_user_step(message.chat.id) == 1)
def msg_voice_select(m):
    cid = m.chat.id
    text = m.text
    if text == 'Записать свой':  
        # ран голоса
        bot.send_chat_action(cid, "record_audio")
        userStep[cid] = 2
        bot.send_message(cid, "Запишите голос которым вы хотите озвучить.")

    elif text in voices:
        bot.send_chat_action(cid, "record_audio")
        userStep[cid] = f"voices/{text}.mp3"
        bot.send_message(cid, "Запишите голосовое или текст который вы хотите озвучить.")

    else:
        bot.send_message(cid, "Please, use the predefined keyboard!")
        bot.send_message(cid, "Please try again")

@bot.message_handler(func=lambda message: type(get_user_step(message.chat.id)) == str)
def generate_voice_message(m):
    cid = m.chat.id
    loaded_file = f'{cid}/message_user_{cid}.ogg'

    if m.content_type == "voice":
        file_info = bot.get_file(m.voice.file_id)
        downloaded_file = bot.download_file(file_info.file_path)
          
        with open(f'{cid}/message_user_{cid}.ogg', 'wb') as new_file:
            new_file.write(downloaded_file)
        
        new_file = f'{cid}/message_user_{cid}.wav'
        subprocess.run(['ffmpeg', '-i', loaded_file, new_file])

        os.remove(loaded_file)
        
        text = recognize_audio(new_file)

    elif m.content_type == "text":
        new_file = userStep[cid]
        text = m.text

    bot.send_message(cid, "Озвучиваем ваше сообщение.")
    audio = generate_audio(new_file, text, loaded_file)
    subprocess.run(['ffmpeg', '-i', loaded_file, '-acodec', 'libopus', loaded_file, '-y'])

    with open(loaded_file, 'rb') as f:
        voice = f.read()
  
    bot.send_message(cid, text)
    bot.send_voice(cid, voice)

    clean(cid)
  
def extract_target_voice(m):

    cid = m.chat.id
    file_info = bot.get_file(m.voice.file_id)
    downloaded_file = bot.download_file(file_info.file_path)
        
    with open(f'{cid}/message_user_{cid}.ogg', 'wb') as new_file:
        new_file.write(downloaded_file)
    
    loaded_file = f'{cid}/message_user_{cid}.ogg'
    new_file = f'voices/message_user_{cid}_target.wav'
    subprocess.run(['ffmpeg', '-i', loaded_file, new_file])
    os.remove(loaded_file)
    userStep[cid] = new_file

    bot.send_message(cid, "Запишите голосовое или текст который вы хотите озвучить.")
  
def clean(cid):
    for _file in os.listdir(str(cid)):
        if _file != ".ipynb_checkpoints":
            os.remove(str(cid) + '/' + _file)

def generate_audio(audio_file_path, text, save_path):
    audio = vcc.run(audio_file_path, text, save_path=save_path)
    return audio
  
def recognize_audio(audio_file_path):
    recognizer = Recognizer()
    with AudioFile(audio_file_path) as source:
        audio = recognizer.record(source)
    text = recognizer.recognize_google(audio, language='RU')
    return text

In [ ]:
import soundfile as sf

class VoiceCloningClass:
    def __init__(self, enc_model_path, syn_model_path, voc_model_path):
        self.device = torch.cuda.current_device()
        self.synthesizer = Synthesizer(Path(syn_model_path).joinpath("taco_pretrained"), low_mem=False)
        self._load_encoder(enc_model_path)
        self._load_vocoder(voc_model_path)

    def _load_encoder(self, enc_model_path):
        inference_encoder.load_model(Path(enc_model_path))

    def _load_vocoder(self, voc_model_path):
        inference_vocoder.load_model(Path(voc_model_path))

    def process_target_voice(self, source_wav_path):
        target_wav, sampling_rate = librosa.load(source_wav_path)
        preprocessed_wav = inference_encoder.preprocess_wav(target_wav, sampling_rate)
        embed, partial_embeds, _ = inference_encoder.embed_utterance(preprocessed_wav, return_partials=True)
        embed /= np.linalg.norm(embed)
        return embed

    def synthesize_voice(self, text, voice_embed):
        texts = g2p([text])
        specs = self.synthesizer.synthesize_spectrograms(texts, [voice_embed])
        generated_wav = inference_vocoder.infer_waveform(specs[0])
        generated_wav = np.pad(generated_wav, (0, self.synthesizer.sample_rate), mode="constant")
        return generated_wav

    def run(self, source_wav_path, text, save_path=None):
        voice_embed = self.process_target_voice(source_wav_path)
        generated_wav = self.synthesize_voice(text, voice_embed)
        if save_path:
            self.save_wav(save_path, generated_wav)
        return generated_wav


    def save_wav(self, path, wav):
      # sf.write(path, wav, self.synthesizer.sample_rate, 'PCM_24')
        librosa.output.write_wav(path, wav.astype(np.float32),
                                 self.synthesizer.sample_rate)

In [ ]:
ENC_PATH = "encoder/saved_models/pretrained.pt"
SYNTH_PATH = "synthesizer/saved_models/logs-pretrained/"
VOC_PATH = "vocoder/saved_models/pretrained/pretrained.pt"

vcc = VoiceCloningClass(ENC_PATH, SYNTH_PATH, VOC_PATH)

Found synthesizer "pretrained" trained to step 348000
Loaded encoder "pretrained.pt" trained to step 1617001
Building Wave-RNN
Trainable Parameters: 4.481M
Loading model weights at vocoder/saved_models/pretrained/pretrained.pt


In [ ]:
bot.polling()

Got message of type text
Eskimo [799213094]: /chooseVoice
Got message of type text
Eskimo [799213094]: Записать свой
Got message of type audio
